In [ ]:
url = 'https://raw.githubusercontent.com/AnanyaThyagarajan/Python-Projects/main/Movie%20Recommend/Tamil_movies_dataset.csv'
data_movies = pd.read_csv(url)


In [ ]:
import requests
import tarfile
import os
import email
import email.policy
from email.parser import BytesParser
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pickle

def download__file(url, save_path):
    """Downloads a file from a URL to the specified local path."""
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Failed to download {url}")

def extract__tarfile(file_path, extract_path="."):
    """Extracts a tar.bz2 file to the specified directory."""
    with tarfile.open(file_path, "r:bz2") as file:
        file.extractall(path=extract_path)

def load_emails_from_directory(directory):
    """Loads all emails from the specified directory, parsing them for text content."""
    emails = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as file:
            msg = BytesParser(policy=email.policy.default).parse(file)
            email_body = msg.get_body(preferencelist=('plain', 'html')).get_content()
            emails.append(email_body)
    return emails

In [ ]:


# URLs of your GitHub-hosted files (replace these with the actual URLs)
urls = [
    'https://raw.githubusercontent.com/AnanyaThyagarajan/Spam-Detection/main/Dataset/20030228_spam_2.tar.bz2',
    'https://raw.githubusercontent.com/AnanyaThyagarajan/Spam-Detection/main/Dataset/20050311_spam_2.tar.bz2',
    'https://raw.githubusercontent.com/AnanyaThyagarajan/Spam-Detection/main/Dataset/20030228_easy_ham_2.tar.bz2',
    'https://raw.githubusercontent.com/AnanyaThyagarajan/Spam-Detection/main/Dataset/20030228_hard_ham.tar.bz2'
]
files = ['20030228_spam_2.tar.bz2', '20050311_spam_2.tar.bz2', '20021010_easy_ham.tar.bz2', '20021010_hard_ham.tar.bz2']
directories = ['20030228_spam_2', '20050311_spam_2', '20021010_easy_ham', '20021010_hard_ham']

# Download and extract files
for url, file, directory in zip(urls, files, directories):
    download__file(url, file)
    extract__tarfile(file, directory)

# Load emails and label them
data_frames = []
for directory, label in zip(directories, [1, 1, 0, 0]):  # spam = 1, ham = 0
    emails = load_emails_from_directory(directory)
    df = pd.DataFrame(emails, columns=['message'])
    df['label'] = label
    data_frames.append(df)

# Combine datasets
combined_data = pd.concat(data_frames, ignore_index=True)

# Split data
X_train, X_test, y_train, y_test = train_test_split(combined_data['message'], combined_data['label'], test_size=0.2, random_state=42)

# Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# SVM Model
model = SVC(kernel='linear')
model.fit(X_train_tfidf, y_train)

# Predict and Evaluate
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save model and vectorizer
pickle.dump(model, open('spam_svm_model.pkl', 'wb'))
pickle.dump(vectorizer, open('tfidf_vectorizer.pkl', 'wb'))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import email  # for parsing emails
import pickle

def parse__email(text):
    """Parse email from text to extract plain text content."""
    try:
        msg = email.message_from_string(text)
        for part in msg.walk():
            if part.get_content_type() == 'text/plain':
                return part.get_payload()
    except:
        return text

# Load your dataset here (make sure it's properly formatted with 'label' and 'message' columns)

url = 'https://raw.githubusercontent.com/AnanyaThyagarajan/Python-Projects/main/Movie%20Recommend/Tamil_movies_dataset.csv'
data_ = pd.read_csv(url)
data = pd.read_csv('combined_dataset.csv')
data['message'] = data['message'].apply(parse_email)

# Label encoding (if not already done)
data['label'] = data['label'].map({'ham': 0, 'spam': 1})

# Split data
X_train, X_test, y_train, y_test = train_test_split(data['message'], data['label'], test_size=0.2, random_state=42)

# Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# SVM Model
model = SVC(kernel='linear')
model.fit(X_train_tfidf, y_train)

# Predict and Evaluate
y_pred = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save model and vectorizer
pickle.dump(model, open('spam_svm_model.pkl', 'wb'))
pickle.dump(vectorizer, open('tfidf_vectorizer.pkl', 'wb'))
